In [9]:
# A neural stack can learn to correctly accept a sequence of inputs, remember them,
# and transorm them according to a pattern learnt from data.

# How a Neural Stack Learns:
# - Accepting input data, pushing & popping according to the patterns learnt by the NN
# - Compares input + output data to account for the accuracy error
# - Use backprop to update the Net into making more accurate predictions.

In [10]:
import numpy as np

stack_width = 3
copy_length = 5

# Vector0
v_0 = np.zeros(stack_width)
print "v_0", v_0
v_0[0] = 1
# Vector1
v_1 = np.zeros(stack_width)
print "v_1", v_1
v_1[1] = 1
# Vector2
v_2 = np.zeros(stack_width)
print "v_2", v_2
v_2[2] = 1

# INIT
# The list of vectors
V = list()
# A list that holds the strengths of vectors
s = list()
# push strength list
d = list()
# pull strength list
u = list()

# The overall vector represntation
def r_t(t):
    # A zero stack of stack width 3
    r_t_out = np.zeros(stack_width)
    # going from range 0 to t+1(since we start t=0, we have to go up till t+1)
    for i in xrange(0,t+1):
        temp = min(s[t][i],max(0,1 - sum(s[t][i+1:t+1])))
        r_t_out += temp * V[t][i]
    print "r_t_out", r_t_out
    return r_t_out

# The strength vector.
def s_t(i,t,u,d):
    if(i >= 0 and i < t):
        inner_sum = s[t-1][i+1:t]
        return max(0,s[t-1][i] - max(0,u[t] - sum(inner_sum)))
    elif(i == t):
        return d[t]
    else:
        print("Undefined i -> t relationship")

def pushAndPop(v_t,d_t,u_t,t=len(V)):

  d.append(d_t)
  u.append(u_t)

  new_s = np.zeros(t+1)
  for i in xrange(t+1):
      new_s[i] = s_t(i,t,u,d)
  s.append(new_s)
  
  if(len(V) == 0):
      V_t = np.zeros((1,stack_width))
      V_t += v_t
  else:
      depth = len(V[-1])
      V_t = np.zeros((depth+1,stack_width))
      for i in xrange(depth):
        V_t[i] += V[-1][i]
      V_t[depth] += v_t
  
  V.append(V_t)
  return r_t(t)

print str(pushAndPop(v_0,0.8,0.0,0))
print str(pushAndPop(v_1,0.5,0.1,1))
print str(pushAndPop(v_2,0.9,0.9,2))

# Stack is empty again
V = list() # stack states
s = list() # stack strengths 
d = list() # push strengths
u = list() # pop strengths

assert str(pushAndPop(v_0,0.8,0.0,0)) == str((0.8 * v_0))
assert str(pushAndPop(v_1,0.5,0.1,1)) == str((0.5 * v_0) + (0.5 * v_1))
assert str(pushAndPop(v_2,0.9,0.9,2)) == str((0.9 * v_2) + (0 * v_1) + (0.1 * v_0))

print "\nFinal Value of S:"
for i in range(3):
  print(s_t(2-i,2,u,d))

print("\nPassed All Assertions!!!")

v_0 [ 0.  0.  0.]
v_1 [ 0.  0.  0.]
v_2 [ 0.  0.  0.]
r_t_out [ 0.8  0.   0. ]
[ 0.8  0.   0. ]
r_t_out [ 0.5  0.5  0. ]
[ 0.5  0.5  0. ]
r_t_out [ 0.1  0.   0.9]
[ 0.1  0.   0.9]
r_t_out [ 0.8  0.   0. ]
r_t_out [ 0.5  0.5  0. ]
r_t_out [ 0.1  0.   0.9]

Final Value of S:
0.9
0
0.3

Passed All Assertions!!!


In [11]:
# We define a vector that is s_t(strength and it takes in the params):
# i = the index of the row
# t = the timestep
# u = pop strength
# d = push strength

def s_t(i,t,u,d):
    if(i >= 0 and i < t):
        # This is the sum of the layer strengths between i+1 & t which is the top of the stack.
        inner_sum = sum(s[t-1][i+1:t])
        out = max(0, s[t-1][i] - max(0, u[t]-(inner_sum)))
        return out
    elif (i == t):
        return d[t]
    else:
        print "Undefined"

def r_t(t):
    r_t_out = np.zeros(stack_width)
    for i in xrange(0, t+1):
        temp = min(s[t][i], max(0,1-sum(s[t][i+1:t+1])))
        r_t_out += temp * V[t][i]
    return r_t_out

In [12]:
def s_t_err(i, t, u, d, err):
    if(i >= 0 and i < t):
        if(s_t(i,t,u,d) >= 0):
            s_delta[t-1][i] += err
            if(max(0, u[t] - np.sum(s[t-1][i+1:t-0])) >= 0):
                u_delta[t] = err
                s_delta[t-1][i+1:t-0] += err
            elif(i == t):
                d_delta[t] += err
            else:
                print "WE GOT A PROBLEM}"


In [ ]:
import numpy as np

u_weights = np.array([0,0,0,1,0.5,1])
d_weights = np.array([1.0,1.0,1,0,0,0])
stack_width = 2
copy_length = 5



# INIT
V = list() # stack states
s = list() # stack strengths 
d = list() # push strengths
u = list() # pop strengths

V_delta = list() # stack states
s_delta = list() # stack strengths 
d_delta = list() # push strengths
u_delta = list() # pop strengths


def r_t(t):
  r_t_out = np.zeros(stack_width)
  for i in xrange(0,t+1):
    temp = min(s[t][i],max(0,1 - sum(s[t][i+1:t+1])))
    r_t_out += temp * V[t][i]
  return r_t_out
  
def r_t_error(t,r_t_error):
  for i in xrange(0, t+1):
    temp = min(s[t][i],max(0,1 - sum(s[t][i+1:t+1])))
    V_delta[t][i] += temp * r_t_error
    temp_error = sum(r_t_error * V[t][i])

    if(s[t][i] < max(0,1 - sum(s[t][i+1:t+1]))):
        s_delta[t][i] += temp_error
    else:
        if(max(0,1 - sum(s[t][i+1:t+1])) > 0):
          for j in xrange(t-i):
            s_delta[t][i+j+1] -= temp_error # minus equal becuase of the (1-).. and drop the 1
    
def s_t(i,t,u,d):
    if(i >= 0 and i < t):
        inner_sum = s[t-1][i+1:t]
        return max(0,s[t-1][i] - max(0,u[t] - sum(inner_sum)))
    elif(i == t):
        return d[t]
    else:
        print "Undefined i -> t relationship"
        
def s_t_error(i,t,u,d,error):
    if(i >= 0 and i < t):
        if(s_t(i,t,u,d) >= 0):
            s_delta[t-1][i] += error
            if(max(0,u[t] - sum(s[t-1][i+1:t])) >= 0):
                u_delta[t] -= error
                for j in xrange(t-(i+1)):
                  s_delta[t-1][i+j+1] += error
    elif(i == t):
        d_delta[t] += error
    else:
        print "Problem"
        
        
def pushAndPop(v_t,d_t,u_t,t=len(V),deriv=False,error=None):
    if(not deriv):
        d.append(d_t)
        d_delta.append(0)

        u.append(u_t)
        u_delta.append(0)

        new_s = np.zeros(t+1)
        for i in xrange(t+1):
            new_s[i] = s_t(i,t,u,d)
        s.append(new_s)
        s_delta.append(np.zeros(new_s.shape[0]))

        if(len(V) == 0):
            V_t = np.zeros((1,stack_width))
        else:
            depth = len(V[-1])
            V_t = np.zeros((depth+1,stack_width))
            for i in xrange(depth):
              V_t[i] += V[-1][i]
            V_t[depth] += v_t
        
        V.append(V_t)
        V_delta.append(np.zeros(V[-1].shape[0],V[-1].shape[1]))

        return r_t(t)
    else:
        r_t_error(t,error)
        for i in xrange(t+1):
            s_t_error((t+1) - i - 1,t,u,d,s_delta[t][i])
            
for i in xrange(500):
    alpha = 5 * ((1 - (float(i)/500)) **2)

    sequence = np.array([[0.1,0.2,0.3],[0,0,0]]).T

    # RE-INITIALIZE WEIGHTS (empty the stack and strengths)
    V = list() # stack states
    s = list() # stack strengths 
    d = list() # push strengths
    u = list() # pop strengths

    V_delta = list() # stack states
    s_delta = list() # stack strengths 
    d_delta = list() # push strengths
    u_delta = list() # pop strengths

    out_0 = pushAndPop(sequence[0],d_weights[0],u_weights[0],t=0)
    out_1 = pushAndPop(sequence[1],d_weights[1],u_weights[1],t=1)
    out_2 = pushAndPop(sequence[2],d_weights[2],u_weights[2],t=2)

    y = np.array([1,0])

    out_3 = pushAndPop(sequence[2],d_weights[3],u_weights[3],t=3)
    out_4 = pushAndPop(sequence[2],d_weights[4],u_weights[4],t=4)
    out_5 = pushAndPop(sequence[2],d_weights[5],u_weights[5],t=5)
    
    pushAndPop(sequence[2],0,u_weights[5],t=5,deriv=True,error=out_5 - (y*0.1))
    pushAndPop(sequence[2],0,u_weights[4],t=4,deriv=True,error=out_4 - (y*0.2))
    pushAndPop(sequence[2],0,u_weights[3],t=3,deriv=True,error=out_3 - (y*0.3))

    u_weights[3] -= alpha * u_delta[3]
    u_weights[4] -= alpha * u_delta[4]
    u_weights[5] -= alpha * u_delta[5]

    if(i % 100 == 0):
        print "\n\nIteration:"
        print i
        print
        # print u_delta
        print out_3[0]
        print out_4[0]
        print out_5[0]